# Docker/Container Class

#### Made by: SSG McCracken

## Information

### What is a Container

### What is Docker

## Practical Excersices

#### Environment Setup Script

In [ ]:
!apt-get -qq update && apt-get -q upgrade -y && apt-get -qq remove -y  docker docker-engine docker.io containerd runc && apt-get -qq install -y docker.io && echo "Done"

In [ ]:
!systemctl is-active docker

In [ ]:
!docker stop `docker ps -aq`; docker rm `docker ps -aq`; docker rmi `docker images -aq`

___

### Docker Commands

___

#### PS

In [ ]:
!docker ps -h

In [ ]:
!docker ps -a

>In the command above we see that we have no containers in any status running. This is because we removed all containers in our initial setup above with the command below:
> 
> ``docker stop `docker ps -aq` && docker rm `docker ps -aq` ``
>
> An example of all of these commands will will be below

> Lets start a container to show what a running container looks like in `docker ps`

In [ ]:
!docker run hello-world

> Below we can show all containers with `docker ps -a`

In [ ]:
!docker ps -a

> Lets start a container that will run indefinetly to show what a running container looks like with `docker ps`

In [ ]:
!docker run -d ubuntu sleep infinity

> Below you will see three different commands
>
> `docker ps` shows the running container
>
> `docker ps -a` shows all containers
>
> `docker ps -aq` shows all containers but only with the container id
>
>> `docker ps -aq` is useful for piping all containers to commands like `docker rm `docker ps -aq` as you will see me use throughout this lesson

In [ ]:
!docker ps

In [ ]:
!docker ps -a

In [ ]:
!docker ps -aq

> Below you will see me stop all containers and remove them using `docker ps -aq`

In [ ]:
!docker stop `docker ps -aq` && docker rm `docker ps -aq`

> Just to verify that they are all removed we will run `docker ps -a`

In [ ]:
!docker ps -a

___

#### Docker Images

In [ ]:
!docker images -h

In [ ]:
!docker images -a

> Below I will use `docker -rmi <container name>` to remove a specific image

In [ ]:
!docker rmi hello-world

> I'll run `docker images -a` just to verify it was removed

In [ ]:
!docker images -a

> I'll use `docker pull <image name>` to pull an image to my local machine but not run it

In [ ]:
!docker pull hello-world

In [ ]:
!docker images -a

___
#### Run

In [ ]:
!docker run --help

> Above I used `docker run -d ubuntu sleep infinity` to start a ubuntu image and execute the `sleep inifinity` command in and I used the `-d` to run it in a detached status.
> I'll do it again below

In [ ]:
!docker run -d --name will_not_die ubuntu sleep infinity

> So now we have a ubuntu container that has a process running. While the process is running the container will stay alive. Once the process completes the docker container wil exit

> Let me demonstrate that with the command below.
>> NOTE: This container will be ran in the "attached" mode.

In [ ]:
!docker run --name will_die ubuntu sleep 5

In [ ]:
!docker ps -a

> Above you can see that the "will_die" container exited, while the "will_not_die" is still running

> We can attach to the container by attaching to it using `docker attach will_not_die` or `docker attach <container_id>`

> We can map ports using `-p host_port:container_port`

>> Note: `--rm` auto removes a docker container after it shuts down

In [ ]:
!docker run --name port_test -d --rm -p 8080:80 yeasy/simple-web

In [ ]:
!docker stop port_test

___
#### EXEC

> We can also run a command on a container by using `docker exec`

In [ ]:
!docker exec will_not_die env

> We can also use this to get a terminal to the container using `docker exec -it <name> /bin/bash`

___
#### Volumes

> Volumes are how you create persistance between docker containers, upload data to a docker, and share information between dockers

> `docker run -v host_location:container_location ...`

>> Note: Docker will create the directories needed on either the host or the container

In [ ]:
!mkdir test_dir && touch test_dir/test.txt && ls test_dir/

In [ ]:
!docker run --rm -v ~/docker_class/test_dir:/tmp/docker ubuntu ls /tmp/docker

In [ ]:
!docker run --rm -v ~/docker_class/new_dir:/logs ubuntu /bin/bash -c 'echo "This is a test" >> /logs/log.txt'

In [ ]:
!cat new_dir/log.txt

In [ ]:
!rm -rf new_dir test_dir

___
#### INSPECT


> docker inspect gives you more information than what ps will

In [ ]:
!docker run -d --name inspect_test --rm -v ~/docker_class/test_dir:/logs ubuntu sleep infinity

In [ ]:
!docker inspect inspect_test

In [ ]:
!docker stop inspect_test

___
#### Environment Variables

In [ ]:
script = """
import os

variable = os.environ.get("TEST_VAR")

print(f"This is the variable I was given = {variable}")
"""

In [ ]:
with open("print_var.py", "w") as f:
    f.write(script)

In [ ]:
!python3 print_var.py

In [ ]:
!docker run --rm -v "$PWD":/usr/src/myapp -w /usr/src/myapp python:3 python print_var.py

In [ ]:
!docker run -e TEST_VAR="THIS IS MY TEXT" --rm -v "$PWD":/usr/src/myapp -w /usr/src/myapp python:3 python print_var.py

___
#### Networks

> Docker networks allows you to network containers together.
> Networks will provide dns and dhcp
> Docker container addresses will be thier name

In [ ]:
!docker network --help

In [ ]:
!docker network ls

In [ ]:
!docker network create my_net

In [ ]:
!docker network ls

In [ ]:
!docker run -d --rm --network my_net --name rec_host ubuntu sleep infinity

In [ ]:
!docker run --rm --network my_net --name send_host busybox ping -c 2 rec_host

___
## Dockerfiles
___

> Docker hub is essentially docker github

> A Dockerfile is used to recreate everything you can do using the docker run command

In [ ]:
doc_file = """
FROM python:3

ENV TEST_VAR="Hi"

COPY print_var.py /usr/src/myapp/
WORKDIR /usr/src/myapp/

ENTRYPOINT python print_var.py
"""

In [ ]:
with open("Dockerfile", "w") as f:
    f.write(doc_file)

In [ ]:
!docker build -t variable_printer .

In [ ]:
!docker images

In [ ]:
!docker run --rm variable_printer

In [ ]:
!mkdir var_printer

In [ ]:
!mv Dockerfile print_var.py var_printer/.

___
docker-compose
___

> docker-compose is a method of deploying multiple services (i.e. dockerfiles) using one yml

In [ ]:
doc_comp = """
version: "3"
services:
    var_printer:
        build: ./var_printer
"""

In [ ]:
with open("docker-compose.yml", "w") as f:
    f.write(doc_comp)

In [ ]:
!docker-compose up

In [ ]:
!mkdir ping_rec ping_send

In [ ]:
ping_rec = """
FROM ubuntu

ENTRYPOINT sleep infinity
"""

In [ ]:
ping_send = """
FROM busybox

ENTRYPOINT ping ping_rec -c 20
"""

In [ ]:
with open("ping_send/dockerfile", "w") as f:
    f.write(ping_send)

In [ ]:
with open("ping_rec/dockerfile", "w") as f:
    f.write(ping_rec)

In [ ]:
doc_comp = """
version: "3"
services:
    ping_send:
        build: ./ping_send
    ping_rec:
        build: ./ping_rec
"""

In [ ]:
with open("docker-compose.yml", "w") as f:
    f.write(doc_comp)

In [ ]:
!docker-compose build

In [ ]:
!docker-compose up

In [ ]:
!docker-compose down

In [ ]:
ping_send = """
FROM ubuntu

RUN apt-get -qq update
RUN apt-get -qq install iputils-ping

ENTRYPOINT ping ping_rec -c 20
"""

In [ ]:
with open("ping_send/dockerfile", "w") as f:
    f.write(ping_send)

In [ ]:
!docker-compose build

In [ ]:
!docker-compose up

In [ ]:
!docker-compose down

___
# Assignment

> Make the docker-compose and dockerfile for a server, and 5 clients of the below servers. This server was not my code ... don't judge me.
>
> The Logging file of the server must be persistent between containers

In [ ]:
!rm -rf ping_* var_printer docker-compose.yml

In [ ]:
server = """
import multiprocessing
import socket

def handle(connection, address):
    import logging
    logging.basicConfig(level=logging.DEBUG)
    logger = logging.getLogger("process-%r" % (address,))
    try:
        logger.debug("Connected %r at %r", connection, address)
        while True:
            data = connection.recv(1024)
            if data == "":
                logger.debug("Socket closed remotely")
                break
            logger.debug("Received data %r", data)
            connection.sendall(data)
            logger.debug("Sent data")
    except:
        logger.exception("Problem handling request")
    finally:
        logger.debug("Closing socket")
        connection.close()

class Server(object):
    def __init__(self, hostname, port):
        import logging
        self.logger = logging.getLogger("server")
        self.hostname = hostname
        self.port = port

    def start(self):
        self.logger.debug("listening")
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.bind((self.hostname, self.port))
        self.socket.listen(1)

        while True:
            conn, address = self.socket.accept()
            self.logger.debug("Got connection")
            process = multiprocessing.Process(target=handle, args=(conn, address))
            process.daemon = True
            process.start()
            self.logger.debug("Started process %r", process)

if __name__ == "__main__":
    import logging
    logging.basicConfig(filename="main.log", level=logging.DEBUG)
    logging.getLogger().addHandler(logging.StreamHandler())
    server = Server("0.0.0.0", 9000)
    try:
        logging.info("Listening")
        server.start()
    except:
        logging.exception("Unexpected exception")
    finally:
        logging.info("Shutting down")
        for process in multiprocessing.active_children():
            logging.info("Shutting down process %r", process)
            process.terminate()
            process.join()
    logging.info("All done")
"""

In [ ]:
client = """
import socket
import os
from random import randint
from time import sleep

if __name__ == "__main__":
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect(("localhost", 9000))
    while(True):
        data = socket.gethostname()
        if not data:
            data = "no_data"
        sock.send(data.encode())
        result = sock.recv(1024)
        print(result)
        sleep(randint(1,20))
    sock.close()
"""

In [ ]:
!mkdir server client

In [ ]:
!chmod -R 777 server client

In [ ]:
with open("server/server.py", "w") as f:
    f.write(server)

In [ ]:
with open("client/client.py", "w") as f:
    f.write(client)